# 0.0 Imports

In [1]:
import pickle
import numpy    as np
import pandas   as pd
import seaborn  as sns

from matplotlib import pyplot as plt

from IPython.display import HTML

from sklearn import preprocessing as pp
from sklearn import model_selection as ms

## 0.1 Help functions

In [34]:
# Supressão de Notação Científica
np.set_printoptions(suppress=True)
pd.set_option('display.float_format', '{:.4f}'.format)

In [35]:
# Corrigir Gráficos Jupter

def jupyter_settings():
    %matplotlib inline

    plt.style.use( 'bmh' )
    plt.rcParams['figure.figsize'] = [16, 12]
    plt.rcParams['font.size'] = 12

    display( HTML( '<style>.container { width:100% !important; }</style>') )
    pd.options.display.max_columns = None
    pd.options.display.max_rows = None
    pd.set_option( 'display.expand_frame_repr', False )

jupyter_settings()

## 0.2 Load interim data

In [2]:
df4 = pickle.load( open( "../data/interim/df2_feature.pkl", "rb" ) )
df4.head()

,id,genero,idade,codigo_regiao,contato_cliente,cnh,idade_veiculo,veiculo_danificado,seguro_previo_automovel,premio_anual,cliente_dias_contrato,resposta
0,1,Male,44,28.0,26.0,1,3,1,0,40454.0,217,1
1,2,Male,76,3.0,26.0,1,2,0,0,33536.0,183,0
2,3,Male,47,28.0,26.0,1,3,1,0,38294.0,27,1
3,4,Male,21,11.0,152.0,1,1,0,1,28619.0,203,0
4,5,Female,29,41.0,152.0,1,1,0,1,27496.0,39,0


# 1.0 Data Preparation

In [3]:
df4.columns

Index(['id', 'genero', 'idade', 'codigo_regiao', 'contato_cliente', 'cnh',
       'idade_veiculo', 'veiculo_danificado', 'seguro_previo_automovel',
       'premio_anual', 'cliente_dias_contrato', 'resposta'],
      dtype='object')

## 1.1 Normalização

In [4]:
ss = pp.StandardScaler()

# premio_anual
df4['premio_anual'] = ss.fit_transform( df4[['premio_anual']].values)

In [6]:
pickle.dump(ss,open('../src/features/premio_anual_normalize.pkl','wb'))

## 1.2 Reescala

In [7]:
mms_idade = pp.MinMaxScaler()
mms_cdc = pp.MinMaxScaler()

# idade
df4[['idade']] = mms_idade.fit_transform( df4[['idade']].values )

# cliente_dias_contrato

df4[['cliente_dias_contrato']] = mms_cdc.fit_transform( df4[['cliente_dias_contrato']].values )

In [8]:
pickle.dump( mms_idade, open('../src/features/mms_idade.pkl ','wb' ) )
pickle.dump( mms_cdc, open('../src/features/mms_cdc.pkl','wb' ) )

## 1.3 Encoder

In [71]:
# cnh - tem ou não algo 0 1 - já está em LabelEncoder
# seguro_previo_automovel - já está em LabelEncoder
# veiculo_danificado - já está em LabelEncoder

# genero - OneHotEncoder / TargetEncoder
target_encode_genero = df4.groupby(['genero'])['resposta'].mean()
df4.loc[:, 'genero'] = df4['genero'].map(target_encode_genero)

# codigo_regiao -  One Hot Encoding / Frequency Encoding / Target Encoding / Weighted Target Encoding
target_encode_codigo_regiao = df4.groupby(['codigo_regiao'])['resposta'].mean()
df4.loc[:, 'codigo_regiao'] = df4['codigo_regiao'].map(target_encode_codigo_regiao)

# idade_veiculo - One Hot Encoding / Order Encoding / Frequency Encoding
df4 = pd.get_dummies(df4, prefix=['idade_veiculo'], columns=['idade_veiculo'])

# contato_cliente - Target Encoding / Frequency Encoding
fe_contato_cliente = df4.groupby(['contato_cliente']).size() / len(df4)
df4.loc[:, 'contato_cliente'] = df4['contato_cliente'].map(fe_contato_cliente)

In [ ]:
pickle.dump( target_encode_genero, open('../src/features/target_encode_genero.pkl', 'wb' ) )
pickle.dump( target_encode_codigo_regiao, open('../src/features/target_encode_codigo_regiao.pkl', 'wb' ) )
pickle.dump( fe_contato_cliente , open('../src/features/fe_contato_cliente.pkl', 'wb' ) )

## 1.4 Separação Treino, Validação

In [77]:
X = df4.drop( columns=['id','resposta'], axis=1 )
y = df4['resposta'].copy()

x_treino, x_validacao, y_treino, y_validacao = ms.train_test_split(X, y, test_size=0.2, random_state=42)

# 2.0 Export Pickle interim Data

In [82]:
# x_treino
pickle.dump(x_treino,open('../data/interim/x_treino.pkl','wb'))
# y_treino
pickle.dump(y_treino,open('../data/interim/y_treino.pkl','wb'))

# x_validacao
pickle.dump(x_validacao,open('../data/interim/x_validacao.pkl','wb'))
# y_validacao
pickle.dump(y_validacao,open('../data/interim/y_validacao.pkl','wb'))

In [ ]:
del df4, X, y, x_treino, x_validacao, y_treino, y_validacao